In [2]:
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

#设置gpu设备
device = 'cpu' #f'cuda:{device}' if torch.cuda.is_available() else 
device = torch.device(device)

In [6]:
path='./datasets/632d74d4e2843a53167ee9a1-momodel/' #数据保存路径
save_dir='./results/' #模型保存路径
dataset_name='DGraph'
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor(remove_edge_index=False))

nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2    #本实验中仅需预测类0和类1

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric() #将有向图转化为无向图


if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  #划分训练集，验证集

train_idx = split_idx['train']
valid_idx = split_idx['valid']
test_idx = split_idx['test']

result_dir = prepare_folder(dataset_name,'mlp')

111


In [8]:
print(data)
print(data.x.shape)  #feature
print(data.y.shape)  #label

Data(x=[3700550, 20], edge_index=[2, 4300999], edge_attr=[4300999], y=[3700550], train_mask=[857899], valid_mask=[183862], test_mask=[183840], adj_t=[3700550, 3700550, nnz=7994520])
torch.Size([3700550, 20])
torch.Size([3700550])


In [9]:
nfeats = data.x.shape[1]

In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(nfeats, 16)
        self.conv2 = GCNConv(16, nlabels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [8]:
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [40]:
# model = GCN()
# # 这里可以加载你的模型
# model.load_state_dict(torch.load('./results/GCN.pt'))

<All keys matched successfully>

In [41]:
#Valid Mask准确率
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.valid_mask] == data.y[data.valid_mask]).sum()
acc = int(correct) / int(len(data.valid_mask))
print(f'Accuracy: {acc}')

Accuracy: 0.9873492075578423


In [45]:
len(data.y[data.valid_mask])

183862

In [38]:
#训练集正确率
int((pred[data.train_mask] == data.y[data.train_mask]).sum())/len(data.train_mask)

RuntimeError: The size of tensor a (2) must match the size of tensor b (857899) at non-singleton dimension 1

In [29]:
torch.save(model.state_dict(), save_dir+'/GCN.pt')

In [12]:
pred = model(data).exp()
pred[4]

NameError: name 'model' is not defined

In [48]:
pred = None

In [49]:
pred = None
def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    model = GCN()
    # 这里可以加载你的模型
    model.load_state_dict(torch.load('./results/GCN.pt'))
    # 模型预测时，测试数据已经进行了归一化处理
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    model.eval()
    global pred
    if pred == None:
        pred = model(data).exp()
    y_pred = pred[node_id]
    
    return y_pred

In [50]:
predict(data,131)

tensor([0.9971, 0.0029], grad_fn=<SelectBackward0>)

In [51]:
data.y[valid_idx]

tensor([1, 1, 1,  ..., 0, 0, 0])

In [65]:
pred[valid_idx][:,1].detach()


tensor([0.0004, 0.0165, 0.0512,  ..., 0.0016, 0.0147, 0.0289])

In [52]:
torch.save(pred,save_dir+'/GCNpredTensor.pt',_use_new_zipfile_serialization=False)

In [66]:
from sklearn.metrics import roc_auc_score,roc_curve
fpr,tpr,threshold = roc_curve(data.y[valid_idx],pred[valid_idx][:,1].detach().numpy())

In [68]:
from sklearn.metrics import auc
auc(fpr,tpr)

0.5830233992847355